In [1]:
import pandas as pd

In [2]:
sheetnames = ['January','February','March','April','May','June','July','August','September','October','November','December']
columnnames = ['ID','Joining Day','Demographic', 'Value'];

We want to stack the tables on top of one another, since they have the same fields in each sheet. We can do this one of 2 ways:

    Drag each table into the canvas and use a union step to stack them on top of one another
    Use a wildcard union in the input step of one of the tables

Some of the fields aren't matching up as we'd expect, due to differences in spelling. Merge these fields together

In [3]:
df = pd.concat([pd.read_excel('New Customers.xlsx', sheet_name=s, header=None, names=columnnames, skiprows=1)
                .assign(Month=s) for s in sheetnames])

In [4]:
df.head()

,ID,Joining Day,Demographic,Value,Month
0,490910,3,Ethnicity,White,January
1,490910,3,Date of Birth,5/23/1981,January
2,490910,3,Account Type,Basic,January
3,369221,18,Ethnicity,Black,January
4,369221,18,Date of Birth,3/4/2019,January


In [5]:
df.shape

(2970, 5)

In [6]:
# Make a Joining Date field based on the Joining Day, Table Names and the year 2023
df['Joining Date'] = df['Joining Day'].apply(lambda x: str(x))+" "+df['Month']+" "+"2023"
df.head()

,ID,Joining Day,Demographic,Value,Month,Joining Date
0,490910,3,Ethnicity,White,January,3 January 2023
1,490910,3,Date of Birth,5/23/1981,January,3 January 2023
2,490910,3,Account Type,Basic,January,3 January 2023
3,369221,18,Ethnicity,Black,January,18 January 2023
4,369221,18,Date of Birth,3/4/2019,January,18 January 2023


In [7]:
df['Joining Date'] = pd.to_datetime(df['Joining Date'],dayfirst=True)

In [8]:
# Pivot the table to create new columns from row values
df = df.pivot(index=['ID','Joining Date'],columns=['Demographic'],values=['Value'])
df = df.reset_index()

In [9]:
df.columns=['ID','Joining Date','Account Type','Date of Birth','Ethnicity']
df.head()

,ID,Joining Date,Account Type,Date of Birth,Ethnicity
0,100185,2023-05-20,Basic,7/29/1952,Asian
1,101515,2023-04-14,Gold,8/11/1974,Black
2,101744,2023-08-29,Basic,1/21/1945,Asian
3,102704,2023-01-23,Basic,3/9/2000,Black
4,103488,2023-08-28,Basic,9/26/1957,Other


In [10]:
# Remove Duplicates
df = df.groupby(by=['ID','Account Type','Date of Birth','Ethnicity']).min('Joining Date')
df = df.reset_index()

In [11]:
df

,ID,Account Type,Date of Birth,Ethnicity
0,100185,Basic,7/29/1952,Asian
1,101515,Gold,8/11/1974,Black
2,101744,Basic,1/21/1945,Asian
3,102704,Basic,3/9/2000,Black
4,103488,Basic,9/26/1957,Other
...,...,...,...,...
984,994016,Platinum,3/29/1955,Other
985,994289,Gold,5/9/1990,White
986,994611,Basic,6/19/1994,Black
987,995456,Basic,3/5/1975,Other
